In [1]:
import pandas as pd

In [ ]:
# Read population of each county in the USA

# Load production per category per county in the USA

# Load average needs: Vegetables + Grains + Animal Protein. Scale by population.

# Subtract production and needs. There will be surplus and deficit.

# Find a way to match surplus and deficit, and to penalize for the distance.